In [153]:
import pydot
import matplotlib.pyplot as plt
import time
import math
import random
#import networkx as nx


# Step 1

## Uploading the data

In [154]:
def get_data(dot_file_path):
    '''Returns the nodes, edges (both in pydot format), a list with all the nodes and another with all the edges, and also a dic with (edge):weigth
    '''
     #= 'Datasets/LesMiserables.dot'
    with open(dot_file_path, 'r') as dot_file:
        dot_data = dot_file.read()
    graph = pydot.graph_from_dot_data(dot_data)

    nodes = graph[0].get_nodes()
    edges = graph[0].get_edges()
    
    node_list = [node.get_name() for node in nodes]
    edge_list = [(edge.get_source(), edge.get_destination()) for edge in edges]
    weights = {}
    if edges[0].get_attributes()!={}:
        name= str(list(edges[0].get_attributes().keys())[0])
        weights = {(edge.get_source(), edge.get_destination()):float(edge.get_attributes()[name]) for edge in edges}

    return nodes,edges,node_list,edge_list,weights

# def get_all_weights(edges):
#     weights = {}
#     for edge in edges:
#         weights.append(edge.get_attributes())
    
#     return weights

#
# ans = get_data('Datasets/JazzNetwork.dot')
# print(ans[-1])
#nodes,edges,node_list,edge_list,weights = get_data('Datasets/LesMiserables.dot')
#print(('1','40') in edge_list)
# print(len(node_list)**3) #nodes
# print(len(edge_list))
# print(len(edge_list)*len(node_list)) #edges
# print()
# Nodes:  77 
#Edges:  254
#Total:  33982

In [155]:
# nodes_list0 = get_data('Datasets/JazzNetwork.dot')[2]
# edge_list0 = get_data('Datasets/JazzNetwork.dot')[3]

In [156]:
# print(len(nodes_list0)) #198
# print(len(edge_list0)) #2742

In [157]:
# nodes_list1 = get_data('Datasets/LesMiserables.dot')[2]
# edge_list1 = get_data('Datasets/LesMiserables.dot')[3]

In [158]:
# print(len(nodes_list1)) #77
# print(len(edge_list1)) #254

## Tree Layout

### Counting the number of children/parents for each node

In [159]:
#COUNTING THE NUMBER OF CHILD NODES FOR EACH NODE
def f_childs(file_name_name):  
    '''Returns the count of child nodes [0] and a list of them [1] for all nodes
    ''' 
    nodes = get_data(file_name_name)[0]
    edges = get_data(file_name_name)[1]
    node_child_count = {node.get_name(): 0 for node in nodes}
    node_childs = {node.get_name(): [] for node in nodes}
    for edge in edges:
        source = edge.get_source()
        destination = edge.get_destination()
        if source in node_child_count:
            node_child_count[source] += 1 
            node_childs[source].append(destination) #comment
    return node_child_count, node_childs

#Count
#print(f_childs('Datasets/LesMiserables.dot')[1])
#print(sum(list(f_childs('Datasets/LesMiserables.dot')[0].values())))
#child_11,child_25,child_26 = f_childs('Datasets/LesMiserables.dot')[1]['11'],f_childs('Datasets/LesMiserables.dot')[1]['25'],f_childs('Datasets/LesMiserables.dot')[1]['26']
#print('CHILD')
#print('11: ',child_11)
#print('25: ',child_25)
#print('26": ',child_26)

#List      
#print(f_childs('Datasets/LesMiserables.dot')[1])

def f_parents(file_name_name):
    '''Returns the count of parent nodes [0] and a list of them [1] for all nodes
    '''
    nodes = get_data(file_name_name)[0]
    edges = get_data(file_name_name)[1]
    node_parent_count = {node.get_name(): 0 for node in nodes}
    node_parents = {node.get_name(): [] for node in nodes}
    for edge in edges:
        source = edge.get_source()
        destination = edge.get_destination()
        if destination in node_parent_count:
            node_parent_count[destination] += 1 
            node_parents[destination].append(source)
    return node_parent_count, node_parents

#Count
#print(f_parents('Datasets/LesMiserables.dot')[0]['11'])
#parent_27 = f_parents('Datasets/LesMiserables.dot')[1]['27']
#print("PARENT")
#print('27: ', parent_27)
#List      
#print(f_parents('Datasets/LesMiserables.dot')[1])

def f_edges(file_name):
    '''Returns the count of edges [0] and a list of them [1] for all nodes
    '''
    nodes = get_data(file_name)[0]
    edges = get_data(file_name)[1]
    node_edges_count = {node.get_name(): 0 for node in nodes}
    node_edges = {node.get_name(): [] for node in nodes}
    
    parents = f_parents(file_name)[1]
    childs = f_childs(file_name)[1]

    for node,l_parents in parents.items():
        for parent in l_parents:
            node_edges_count[node] +=1
            node_edges[node].append(parent)
    
    for node,l_childs in childs.items():
        for child in l_childs:
            node_edges_count[node] +=1
            node_edges[node].append(child)
    return node_edges_count,node_edges

#print(f_edges('Datasets/LesMiserables.dot')[1])
#print(f_parents('Datasets/LesMiserables.dot')[1])

#edges = f_edges('Datasets/LesMiserables.dot')
#parents = f_parents('Datasets/LesMiserables.dot')

#print(edges == parents)
#print(edges[0])

### Organizing the nodes throughout the graph, getting their coordinates for the tree layout

In [160]:
def tree_layout(file):
    '''First layout we implemeted
    '''
    total = 0
    n=0
    graph = get_data(file)
    nodes = graph[0]
    edges = graph[1]
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes= []
    node_child_count = f_childs(file)[0]
    node_childs = f_childs(file)[1]
    #lonely_nodes_y = -4
    
    for edge in edges:
        n+=1
        l_m = []
        m=0
        #print(edge)#.get_source())
    #    print(node_childs[str(edge.get_destination())])
        parent_x = node_positions[str(edge.get_source())][0]
        parent_y = node_positions[str(edge.get_source())][1]
        step = 3#random.choice([2,3])

        #nnodes with only one child
        if len(node_childs[str(edge.get_source())]) == 1 and int(edge.get_destination()) not in done_nodes:# and int(edge.get_destination()) not in done_nodes:
            #print(edge)
            #random.choice([2,3])
            child_x = parent_x + step
            child_y = parent_y
            node_positions[edge.get_destination()] = (child_x, child_y)
            done_nodes.append(int(edge.get_destination()))
        
        #nodes with more than one node
        elif len(node_childs[str(edge.get_source())]) > 1 and int(edge.get_destination()) not in done_nodes:
            if len(node_childs[str(edge.get_source())])%2 != 0:
                max_y = (node_child_count[str(edge.get_source())]-1)*3
            else:
                max_y = (node_child_count[str(edge.get_source())]-1)*3+1
            for child in node_childs[str(edge.get_source())]:
                m+=1
                child_x = parent_x + step
                child_y = parent_y + max_y
                node_positions[child] = (child_x, child_y)
                max_y -=8
                done_nodes.append(int(edge.get_destination()))
            l_m.append(m)
            total += m
        total+=n
        #elif len(node_childs[str(edge.get_source())]) == 0 and len(node_childs[str(edge.get_destination())]) == 0:
        #    node_positions[child]

        #elif len([done_node for done_node in done_nodes if done_node == edge.get_destination()])==1:
        #    child_x += 1               
        
    return node_positions,n,l_m,total

#pos,n,m,total = tree_layout('Datasets/LesMiserables.dot')
#print("Edges: ",n, '\n',"Nodes: ",m, '\n',"Total: ",total,)
#f_childs(nodes,edges)[1]['17']

### Improved version of the Tree layout

Get the nodes that have the same parent and are directly connected to each other

In [161]:
def common_parents(file_name):
    '''Returns a dic with all the nodes as keys and as values the number of common parents
    '''
    nodes,edges,node_list,edge_list,weights = get_data(file_name)
    ans = {node:[] for node in node_list}
    count = {node:0 for node in node_list}
    for edge1 in edges:
        for edge2 in edges:
            if edge1.get_source() == edge2.get_source() and edge1.get_destination() != edge2.get_destination() and (str(edge1.get_destination()),str(edge2.get_destination())) in edge_list:# and str(edge1.get_destination()) not in ans:
                #ans.append((str(edge1.get_destination()),str(edge2.get_destination())))
                #ans[edge1.get_source()] = []
                ans[edge1.get_source()].append((str(edge1.get_destination()),str(edge2.get_destination())))
   
    for node,edges in ans.items():
        for edge in edges:
            #print(edge)
            count[edge[1]] += 1
    return count

#common_parents('Datasets/LesMiserables.dot')

In [162]:
def tree_layout_improv(file):
    '''returns the coordinates for each node with the improvment of not having connected siblings in the same vertical line
    '''
    graph = get_data(file)
    nodes = graph[0]
    edges = graph[1]
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes= []
    node_child_count = f_childs(file)[0]
    node_childs = f_childs(file)[1]
    
    for edge in edges:
        parent_x = node_positions[str(edge.get_source())][0]
        parent_y = node_positions[str(edge.get_source())][1]
        
        #weights
        position = str(str(edge.get_attributes())).rfind(':')
        if int(position) != -1:
            w = int(str(edge.get_attributes())[position+3:-2])
            if w != 1:
                w = w/10

        #nodes with only one child
        if len(node_childs[str(edge.get_source())]) == 1 and int(edge.get_destination()) not in done_nodes:# and int(edge.get_destination()) not in done_nodes:
            #print(edge)
            step = 3#random.choice([2,3])
            child_x = parent_x + step
            child_y = parent_y
            node_positions[edge.get_destination()] = (child_x, child_y)
            done_nodes.append(int(edge.get_destination()))
            print()
        
        #nodes with more than one node
        elif len(node_childs[str(edge.get_source())]) > 1 and int(edge.get_destination()) not in done_nodes:
            if len(node_childs[str(edge.get_source())])%2 != 0:
                max_y = (node_child_count[str(edge.get_source())]-1)*3
            else:
                max_y = (node_child_count[str(edge.get_source())]-1)*3+1
            for child in node_childs[str(edge.get_source())]:
                step = 3#random.choice([2,3])
                child_x = parent_x + step
                child_y = parent_y + max_y
                node_positions[child] = (child_x, child_y)
                max_y -=8
                done_nodes.append(int(edge.get_destination()))
        #elif len([done_node for done_node in done_nodes if done_node == edge.get_destination()])==1:
        #    child_x += 1
                
    #correcting the child_x in cases where sibling nodes are connected to each other
    for node,count in [(node,count) for node,count in common_parents(file).items() if count != 0]:
        node_positions[node] = (node_positions[node][0]+count/3,node_positions[node][1])
    
    return node_positions

#tree_layout_improv('Datasets/LesMiserables.dot')

### Time complexity of the improved version

In [163]:
def time_complexity(file,layout,tree='BFS',ite=10):
    '''Returnt the average (ite) time complexity of a specific layout'''
    ans = []
    for i in range(ite):
        if layout == 'tree_layout':
            start_time = time.time() 
            result = tree_layout(file)
            end_time = time.time() 
        
        if layout == 'tree_layout_improv':
            start_time = time.time() 
            result = tree_layout_improv(file)
            end_time = time.time() 
        if layout == 'radial':
            start_time = time.time() 
            result = radial(file,tree)
            end_time = time.time() 
         # Record the end time
        elapsed_time = end_time - start_time  # Calculate the elapsed time
        ans.append(elapsed_time)
    print('In average, the layout',layout, 'takes', round(sum(ans)/len(ans), 2),'seconds to implement on the', file,'dataset.') 

#time_complexity('Datasets/LesMiserables.dot','radial',tree='DFS')


### Bonus Task

In [164]:
#HOW TO CREATE THE DEFAULT LAYOUT
def bonus_task(file,layout):
    with open(file, 'r') as dot_file:
        dot_data = dot_file.read()

    # Create a graph object from the .dot data
    graph = pydot.graph_from_dot_data(dot_data)

    # Set the layout algorithm to "dot" (or any other supported layout type)
    graph[0].set_graph_defaults(layout=layout)
    graph[0].set_graph_defaults(layout=layout,)
    # Render the graph with Graphviz
    output_file_path = layout+'.png'
    graph[0].write_png(output_file_path)

# Step 2

### Constructing a BFS tree

In [165]:
def layered_ish(file):
    '''Returns the node positions {node:(x,y) and also the final tree (edges and stuff)
    '''
    total = 0
    n=0
    graph = get_data(file)
    nodes = graph[0]
    #edges = graph[1]
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes = set()
    #node_child_count = f_childs(file)[0]
    node_childs = f_childs(file)[1]
    min_y = 0

    for node in nodes:
        done_nodes.add(str(node.get_name()))
        childs = node_childs[str(node.get_name())]
        parent_x = node_positions[str(node.get_name())][0]
        parent_y = node_positions[str(node.get_name())][1]
        step = 3

        if len(node_childs[str(node.get_name())]) == 1 and str(node_childs[str(node.get_name())][0]) not in done_nodes:# and int(edge.get_destination()) not in done_nodes:
            child_x = parent_x + step
            child_y = parent_y
            node_positions[str(node_childs[str(node.get_name())][0])] = (child_x, child_y)
            done_nodes.add(str(node_childs[str(node.get_name())][0]))
            
        elif len(node_childs[str(node.get_name())]) >1:# and node_childs[str(node.get_name())] not in done_nodes:
            new_childs = len([child for child in node_childs[str(node.get_name())] if child not in done_nodes])
            max_y = (new_childs-1)*3
            
            done_childs = []
            for child in [new_child for new_child in node_childs[str(node.get_name())] if new_child not in done_nodes]:
                done_childs.append(child)
                child_x = parent_x + step
                child_y = parent_y + max_y
                
                node_positions[child] = (child_x, child_y)
                done_nodes.add(str(child))
                max_y -= 6

            if len(done_childs)>0:
                min_y = node_positions[done_childs[-1]][1]
    return node_positions

#write_ordered_dic('Datasets/LesMiserables.dot')
# tree = layered_ish('Datasets/LesMiserables.dot')
# tree

In [166]:
def BFS_tree(file):
    '''Returns the BFS tree (edges and stuff) {parent:[childs]}
    '''
    graph = get_data(file)
    nodes = graph[0]
    done_nodes = set()
    node_childs = f_childs(file)[1]
    tree = {node.get_name(): [] for node in nodes}
    
    for node in nodes:
        done_nodes.add(str(node.get_name()))
        done_childs = []
        for child in [new_child for new_child in node_childs[str(node.get_name())] if new_child not in done_nodes]:
            done_childs.append(child)
            done_nodes.add(str(child))
            tree[node.get_name()].append(str(child))    
    final_tree = {i:j for i,j in tree.items() if j!= []}
    return final_tree

#BFS_tree('ordered_LesMiserables.dot')
#BFS_tree('Datasets/LesMiserables.dot')


### Constructing the DFS tree

In [167]:
def tree_layout_DFS(file):
    #returns the coordinates for each node 
    graph = get_data(file)
    nodes = graph[0]
    edges = graph[1]

    node_positions = {node.get_name(): (0, 0) for node in nodes}
    done_nodes= []
    node_child_count = f_childs(file)[0]
    node_childs = f_childs(file)[1]
    visited_nodes = []
    tree = {node.get_name(): [] for node in nodes}
    return dfs(node_positions, 1, node_childs=node_childs, visited_nodes=visited_nodes, edges=edges, tree=tree)



def dfs(node_positions, node, node_childs, visited_nodes, edges, tree, x=0, y=0):
    visited_nodes.append(node)
    parent_x, parent_y = x, y
    
    step_x = 1  # Horizontal step size
    step_y = 2  # Vertical step size

    nchilds = len(node_childs[str(node)])
    if nchilds == 0:
        return node_positions

    child_y = parent_y - step_y
    total_width = step_x * (nchilds - 1)
    start_x = parent_x - total_width / 2
    
    for i, child in enumerate(node_childs[str(node)]):
        if child not in visited_nodes:
            child_x = start_x + i * step_x
            node_positions[str(child)] = (child_x, child_y)

            tree[str(node)].append(child)
            dfs(node_positions, child, node_childs, visited_nodes, edges, tree, child_x, child_y)
    
    final_tree = {i:j for i,j in tree.items() if j!= []}
    return node_positions, final_tree

#print(tree_layout_DFS('Datasets/LesMiserables.dot')[0])
#show_graph('Datasets/LesMiserables.dot','layered_ish' ,'DFS',(18,18))

In [168]:
#HOW TO PLOT A GENERAL GRAPH
def show_graph(file,layout,tree='BFS', dims=(18,18)):
    #nodes,edges = get_data(file)[0],get_data(file)[1]
    edge_list,weigths = get_data(file)[3],get_data(file)[4]
    if layout == 'tree_layout':
        node_positions = tree_layout(file)
        edges = get_data(file)[1]
    if layout == 'tree_layout_improv':
        node_positions = tree_layout_improv(file)
        edges = get_data(file)[1]

    if layout == 'layered_ish':
        node_positions = layered_ish(file)
    
    
    
    if tree == 'BFS' and layout in ['layered_ish','radial']:
        edges_ish = BFS_tree(file)
        edges = []
        for source,l_dest in edges_ish.items():
            for dest in l_dest:
                edges.append((source,dest))
        if layout == 'radial':
            node_positions = radial(file,tree_type='BFS')
    
    # if tree == 'DFS' and layout in ['layered_ish','radial']:
    #     edges_ish = DFS_tree(file)[1]
    #     #print(edges_ish)
    #     edges = []
    #     for source,l_dest in edges_ish.items():
    #         for dest in l_dest:
    #             edges.append((source,dest))
    if tree == 'DFS' and layout in ['layered_ish','radial']:
        tree_org = tree_layout_DFS(file)
        edges_ish = tree_org[1]
        node_positions = tree_org[0]
        edges = []
        for source,l_dest in edges_ish.items():
            for dest in l_dest:
                edges.append((source,dest))
        if layout == 'radial':
            node_positions = radial(file,tree_type='DFS')

    plt.figure(figsize=(dims[0], dims[1]))
    w = 1
    for edge in edges:
        if layout not in ['layered_ish','radial']:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
        else:
            start_node = edge[0]
            end_node = edge[1]
            new_edge = (start_node,end_node)
            if new_edge not in edge_list:
                new_edge = (end_node,start_node)
            if weigths !={}:
                w = weigths[new_edge]
                if w != 1:
                    w=w/10

        start_pos = node_positions[start_node]
        end_pos = node_positions[end_node]
        plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 

    for node, pos in node_positions.items():
        plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # Use marker='o' for circle markers
        plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        
    plt.axis('off')
    plt.gca().set_facecolor('w')
    plt.savefig('Plots/'+tree+'_'+layout+'_plot.png')
    plt.show()
    
#show_graph('Datasets/LesMiserables.dot','tree_layout1',(18,18))
#show_graph('Datasets/LesMiserables.dot','radial',tree = 'DFS')

# Radial Layout

### Sorting the nodes based on their degree and writing the new tree in a .dot file

In [169]:
def f_edges(file_name):
    '''Returns the number and list of edges for each one of the nodes
    '''
    nodes = get_data(file_name)[0]
    edges = get_data(file_name)[1]
    node_edges_count = {node.get_name(): 0 for node in nodes}
    node_edges = {node.get_name(): [] for node in nodes}
    
    parents = f_parents(file_name)[1]
    childs = f_childs(file_name)[1]

    for node,l_parents in parents.items():
        for parent in l_parents:
            node_edges_count[node] +=1
            node_edges[node].append(parent)
    
    for node,l_childs in childs.items():
        for child in l_childs:
            node_edges_count[node] +=1
            node_edges[node].append(child)
    return node_edges_count,node_edges

#final_edges = f_edges('Datasets/LesMiserables.dot')[1]

In [170]:
def sort_nodes_degree(file):
    '''Returns a sorted dictionary containing the nodes, as keys, and a list of it's childs, as values, ordered based on the degree (> to <)
    '''
    return dict(sorted(f_edges(file)[1].items(), key=lambda item: len(item[1]), reverse=True))

# dic = sort_nodes_degree('Datasets/LesMiserables.dot')
# dic

In [171]:
def write_ordered_dic(file,new_file):
    '''writes a new .dot file but with the nodes ordered based of their degree
    '''
    dic = sort_nodes_degree(file)
    nodes_list = get_data(file)[2]
    edge_list = get_data(file)[3]
    weigths = get_data(file)[4]
    done_nodes = set()

    graph = pydot.Dot(graph_type='graph')

    # for node_ish in nodes_list:
    #     node = pydot.Node(node_ish)
    #     graph.add_node(node)
    
    for node_ish,childs in dic.items():
        node = pydot.Node(node_ish)
        if node_ish not in done_nodes:
            graph.add_node(node)
            done_nodes.add(node_ish)
        for child_ish in childs:
            child = pydot.Node(child_ish)
            if child_ish not in done_nodes:
                graph.add_node(child)
                done_nodes.add(child_ish)
            #graph.add_node(child)
            edge_ish = (str(node_ish), str(child_ish) )
            if edge_ish not in edge_list:
                edge_ish = (str(child_ish), str(node_ish))
            w = int(weigths[edge_ish])
            edge = pydot.Edge(str(node_ish), str(child_ish), weight=w)
            graph.add_edge(edge)
    
    graph.write(new_file)

#write_ordered_dic('Datasets/LesMiserables.dot','example.dot')

### Auxiliary functions for the radial layout

In [172]:
def get_key(dictionary, value):
    '''Return the key of a specific value
    '''
    return next((key for key, val in dictionary.items() if value in val), None)

def count_descendants(tree, node_name):
    'returns the number (int) of descendants of a node'
    if node_name not in tree:
        return 0
    
    descendants = 0
    if node_name in tree:
        children = tree[node_name]
        descendants += len(children)  # Add the direct children count
        for child in children:
            descendants += count_descendants(tree, child)
    return descendants

def count_ancestors(tree, node_name):
    'returns the number (int) of ancestors of a node'
    ancestors = 0
    nodes = tree
    key = get_key(tree,node_name)
    #print('key',key)
    if key != None:
        parent = get_key(tree,node_name)
        ancestors += 1           
        ancestors += count_ancestors(tree, parent)
    return ancestors

def f_layers(file,layout='BFS'):
    '''Organizes the nodes based on the layer their own on a list of lists(index = layer)
    '''
    #if layout == 'BFS':
    #    tree = radial(file)[0]
    layers = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

    if layout == 'BFS':
        tree = BFS_tree(file)
        node_positions = layered_ish(file)
        for node, childs in tree.items():
            for child in childs:
                layer = int(node_positions[child][0]/3)
                layers[layer].append(child)
    if layout == 'DFS':
        tree = tree_layout_DFS(file)[1]
        node_positions = tree_layout_DFS(file)[0]
        for node, childs in tree.items():
            for child in childs:
                layer = -int(node_positions[child][1]/2)
                layers[layer].append(child)
    root = list(tree.keys())[0]
    layers[0].append(root)
    return [layer for layer in layers if layer != []]

#ans_layers =f_layers('Datasets/LesMiserables.dot',layout='DFS')

### Radial Implementation

In [173]:
def radial(file,tree_type = 'BFS',ordesection = False):
    '''returns the nodes\' positions if we apply the radial layout implementation
    '''
    if ordesection != False:
        new_file = write_ordesection_dic(file)
        file = new_file
    data = get_data(file)
    nodes,nodes_list,edge_list,weigths = data[0],data[2],data[3],data[4]
    if tree_type == 'DFS':
        tree = tree_layout_DFS(file)[1]
        layers = f_layers(file,layout='DFS')
    
    elif tree_type == 'BFS':
        tree = BFS_tree(file)
        layers = f_layers(file,layout='BFS')
    node_positions = {node.get_name(): (0, 0) for node in nodes}
    root = list(tree.keys())[0]

    start_angle=0
    for subtree in tree:
        #root = subtree
        angle = 0
        for node in tree[subtree]:
            for i in layers:
                if node in i:
                    layer = layers.index(i)
            #print(layer)
            radius = 3*layer

            parent_x = node_positions[get_key(tree,node)][0]
            parent_y = node_positions[get_key(tree,node)][1]
            #angle for the section for each node
            section = 2*math.pi*(1+count_descendants(tree, subtree))/count_descendants(tree, root)
            #increment between sibling nodes
            increment = 2*math.pi*(count_descendants(tree, node)+1)/count_descendants(tree, root)
            
            if layer > 1 and tree[subtree].index(node)==0:
                #the starting angle must be the parent angle - half of section              
                start_angle = math.acos(parent_x/(3*(layer-1)))-section/2
                if parent_y <0:
                    #acos has domain [0,pi], so we need to fix it where x>pi
                    start_angle = -math.acos(parent_x/(3*(layer-1)))-section/2
                angle+= start_angle
            
            angle+= increment/2
            child_x = math.cos(angle)*radius
            child_y = math.sin(angle)*radius
            angle+= increment/2
            node_positions[node]=(child_x,child_y)
   
    return node_positions


#write_ordesection_dic('Datasets/LesMiserables.dot')
#ans = radial('ordered_LesMiserables.dot',tree_type='DFS')
#radial('Datasets/LesMiserables.dot',tree_type='DFS')


### Exercises

### Take a graph and compute a BFS tree of it


We decided to prune the layout algorithm we came up with on the first Step of this assignment, in order to get the layered layout, not only as the layout to use for all the tasks from Step 2, but also as way to get a representation of our BFS implementation. However, we ended up realizing that this layout is not the best one, given the amount of space wasted. That can be seen in Figure XXX. It's important to mention the fact that this algorithm is not finished, but if we indeed implemented it completely, the amount of wasted space would be even worse.

In [174]:
#write_ordered_dic('Datasets/LesMiserables.dot','ordered_LesMiserables.dot')
#show_graph('Datasets/LesMiserables.dot','layered_ish')

### Take a graph and compute a DFS tree of it

In order to get a DFS, we...

In [175]:
#show_graph('Datasets/LesMiserables.dot','layered_ish',tree = 'DFS')

### Implement radial layout

Time complexity

In [176]:
#time_complexity('Datasets/LesMiserables.dot','radial',tree='BFS',ite=10)

In [177]:
#time_complexity('Datasets/LesMiserables.dot','radial',10)
#time_complexity('ordered_LesMiserables.dot','radial',10)

radial BFS: 8.57

Implementation

We implemented the radial layout both on the Les Misérables network and to Jazz network, both using BFS

In [178]:
#show_graph('Datasets/LesMiserables.dot','radial',tree = 'BFS')

In [179]:
#show_graph('Datasets/JazzNetwork.dot','radial',tree = 'BFS')

In [180]:
#show_graph('Datasets/JazzNetwork.dot','radial',tree = 'DFS')

Comments

This visualization is not great for the Les Misérables network, given that the central node is node 1, and this node only has one incident edge. One way to fix this, is by ordering the nodes in a way that the node in the center is the one with the highest degree.
When it comes to the Jazz network, the BFS representation looks considerably good.

In [181]:
#write_ordered_dic('Datasets/LesMiserables.dot','ordered_LesMiserables.dot')
#show_graph('ordered_LesMiserables.dot','radial')

### Compare the quality of the visualizations for the BFS and DFS

In [182]:
#show_graph('Datasets/LesMiserables.dot','radial',tree = 'BFS')

In [183]:
#show_graph('Datasets/LesMiserables.dot','radial',tree = 'DFS')

We focused on the LesMirables dataset. Both radial layouts, using BFS or DFS trees, have quite decent representations. However, the radial layout with BFS returns a layout where the hierarchical relationships between nodes are more obvious and clear. On the DFS version it's hard to understand where the central node ('1') is actually located, whereas on the BFS it's extremely obvious.
We therefore conclude that the BFS version of this layout algorithm results in better results. 

In [184]:
#show_graph('ordered_LesMiserables.dot','radial',tree = 'BFS')

In [185]:
#nodes,edges,node_list,edge_list,weights = get_data('Datasets/LesMiserables.dot')[0],get_data('Datasets/LesMiserables.dot')[1],get_data('Datasets/LesMiserables.dot')[2],get_data('Datasets/LesMiserables.dot')[3],get_data('Datasets/LesMiserables.dot')[4]

In [186]:
#edge_list

In [187]:
# Example usage
num_nodes = 10
nodes = [i for i in range(0,10)]
edges = [(0, 1), (1, 2), (2, 3), (3, 0), (0, 4), (1, 5), (2, 6), (3, 7), (4, 5), (5, 6), (6, 7), (7, 4), (4, 8), (5, 9)]

def create_pydot_graph(nodes, edges, filename,edge_weights={}):
    graph = pydot.Dot(graph_type='graph')

    # Add nodes to the graph
    for node in nodes:
        graph.add_node(pydot.Node(node))

    # Add edges to the graph
    for edge in edges:
        src, dest = edge
        weight = edge_weights.get(edge, 0)  # Retrieve weight for the edge, defaulting to empty string if not found
        graph.add_edge(pydot.Edge(src, dest, label=str(weight)))
        
        

    # Write the graph to a .dot file
    graph.write(filename)
create_pydot_graph(nodes, edges,'test_force.dot')

# Step 3

In [188]:
def working_spring_embedder_hugo(file, max_ites=1000,convergence_threshold=0.0005, plot='No', c_spring=2, c_rep=1, l=1, step=0.04): 
    nodes, edges, node_list, edge_list, weights = get_data(file)[0], get_data(file)[1], get_data(file)[2], get_data(file)[3], get_data(file)[4]
    node_positions = {node.get_name(): (random.uniform(0, 50), random.uniform(0, 50)) for node in nodes}

    l_changes = []
    ite = 0
    while ite < max_ites:
        changes = []
        for u in nodes:
            force_x = 0
            force_y = 0
            pu = node_positions[u.get_name()]

            for v in nodes:
                if u.get_name() != v.get_name():
                    pv = node_positions[v.get_name()]
                    dist = math.sqrt((pu[0]-pv[0])**2 + (pu[1]-pv[1])**2)
                    if dist == 0:
                        dist = 1

                    # Spring
                    if (u.get_name(), v.get_name()) in edge_list or (v.get_name(), u.get_name()) in edge_list:
                        constant = c_spring*math.log(dist)/l
                        force_x += (pv[0]-pu[0])*constant/dist
                        force_y += (pv[1]-pu[1])*constant/dist
                    # Repulsion
                    else:
                        constant = c_rep/dist**2
                        force_x += (pu[0]-pv[0])*constant/dist
                        force_y += (pu[1]-pv[1])*constant/dist
            changes.append(math.sqrt((force_x*step)**2 + (force_y*step)**2))
            new_x = pu[0] + force_x*step
            new_y = pu[1] + force_y*step
            node_positions[u.get_name()] = (new_x,new_y)

        average_movement = sum(changes) / len(changes)
        l_changes.append(average_movement)
        
        
        if average_movement < convergence_threshold:
            print("convergence signal!!!", ite) 
            break
        
        ite += 1
        step = step*0.99

    if plot != 'No':
        plt.figure(figsize=(18, 18))         
        for edge in edges:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
            start_pos = node_positions[start_node]
            end_pos = node_positions[end_node]
            plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 
        

        for node, pos in node_positions.items():
            plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # vse marker='o' for circle markers
            plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        #plt.axis('off')
        plt.gca().set_facecolor('w')
        #plt.savefig('Plots/'+tree+'_'+layovt+'_plot.png')
        plt.show()
        return None


    return node_positions#, l_changes


#l_f_v = working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000, plot='Yes',c_spring=6)

# Iterations

In [189]:
# working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=250,convergence_threshold=0,plot='yes')
# working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=500,convergence_threshold=0,plot='yes')
# working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=750,convergence_threshold=0,plot='yes')
# working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,convergence_threshold=0,plot='yes')

# Threshold

In [190]:
# working_spring_embedder_hugo('Datasets/LesMiserables.dot', convergence_threshold=0.0005,plot='yes')
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', convergence_threshold=0.0005))

394, 418, 356, 392, 421
AVG Iterations: 396.2

In [191]:
# working_spring_embedder_hugo('Datasets/LesMiserables.dot', convergence_threshold=0.001,plot='yes')
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', convergence_threshold=0.001))

AVG Iterations: 328

In [192]:
# working_spring_embedder_hugo('Datasets/LesMiserables.dot', convergence_threshold=0.005,plot='yes')
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', convergence_threshold=0.005))

191
AVG Iterations: 181

# Ideal spring force constant

In [193]:
# print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,plot='yes',c_spring=2))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=2))

389+400+423+362+375+395
AVG Iterations: 390

In [194]:
# print(working_spring_embedder_hugo('Datasets/LesMiserables.dot',plot= 'Yes', max_ites=1000,c_spring=4))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=4))

ans = 308+365+319+354+315
AVG Iterations: 332

In [195]:
# print(working_spring_embedder_hugo('Datasets/LesMiserables.dot',plot= 'Yes', max_ites=1000,c_spring=6))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=6))

In [196]:
# ans = 308+365+319+354+315
# ans/5


AVG Iterations:

# Ideal rep force constant

In [197]:
#print(working_spring_embedder_hugo('Datasets/LesMiserables.dot',plot= 'Yes', max_ites=1000,c_spring=6,c_rep=1))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=6,c_rep=1))


AVG Iterations:

In [198]:
#print(working_spring_embedder_hugo('Datasets/LesMiserables.dot',plot= 'Yes', max_ites=1000,c_spring=6,c_rep=3))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=6,c_rep=3))


AVG Iterations:

In [199]:
#print(working_spring_embedder_hugo('Datasets/LesMiserables.dot',plot= 'Yes', max_ites=1000,c_spring=6,c_rep=5))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=6,c_rep=5))


AVG Iterations:

# Ideal length

In [200]:
#print(working_spring_embedder_hugo('Datasets/LesMiserables.dot',plot= 'Yes', max_ites=1000,c_spring=6,c_rep=5, l=1))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=6,c_rep=5,, l=1))


AVG Iterations:

In [201]:
#print(working_spring_embedder_hugo('Datasets/LesMiserables.dot',plot= 'Yes', max_ites=1000,c_spring=6,c_rep=5, l=2))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=6,c_rep=5,, l=2))

AVG Iterations:

In [202]:
#print(working_spring_embedder_hugo('Datasets/LesMiserables.dot',plot= 'Yes', max_ites=1000,c_spring=6,c_rep=5, l=3))
# for _ in range(4):
#     print(working_spring_embedder_hugo('Datasets/LesMiserables.dot', max_ites=1000,c_spring=6,c_rep=5,, l=3))


AVG Iterations:

In [203]:
#get_data('Datasets/smallnet.dot')[3]

# Step 4

In [204]:
#HOW TO PLOT A GENERAL GRAPH
def show_graph12(file,layout,tree='BFS', dims=(18,18)):
    #nodes,edges = get_data(file)[0],get_data(file)[1]
    edge_list,weigths = get_data(file)[3],get_data(file)[4]
    if layout == 'tree_layout':
        node_positions = tree_layout(file)
        edges = get_data(file)[1]
    if layout == 'tree_layout_improv':
        node_positions = tree_layout_improv(file)
        edges = get_data(file)[1]
    if layout == 'layered_ish':
        node_positions = layered_ish(file)
        #print(node_positions)
    
    if tree == 'BFS' and layout in ['layered_ish','radial']:
        edges_ish = BFS_tree(file)
        edges = []
        for source,l_dest in edges_ish.items():
            for dest in l_dest:
                edges.append((source,dest))
        if layout == 'radial':
            node_positions = radial(file,tree_type='BFS')
    
    # if tree == 'DFS' and layout in ['layered_ish','radial']:
    #     edges_ish = DFS_tree(file)[1]
    #     #print(edges_ish)
    #     edges = []
    #     for source,l_dest in edges_ish.items():
    #         for dest in l_dest:
    #             edges.append((source,dest))
    if tree == 'DFS' and layout in ['layered_ish','radial']:
        tree_org = tree_layout_DFS(file)
        edges_ish = tree_org[1]
        #node_positions = tree_org[0]
        edges = []
        for source,l_dest in edges_ish.items():
            for dest in l_dest:
                edges.append((source,dest))
        if layout == 'radial':
            node_positions = radial(file,tree_type='DFS')

    plt.figure(figsize=(dims[0], dims[1]))
    w = 1
    for edge in edges:
        if layout not in ['layered_ish','radial']:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
        else:
            start_node = edge[0]
            end_node = edge[1]
            new_edge = (start_node,end_node)
            if new_edge not in edge_list:
                new_edge = (end_node,start_node)
            if weigths !={}:
                w = weigths[new_edge]
                if w != 1:
                    w=w/10

        start_pos = node_positions[start_node]
        end_pos = node_positions[end_node]
        #plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 
        plt.arrow(start_pos[0], start_pos[1],(end_pos[0] - start_pos[0]),(end_pos[1]-start_pos[1]), head_width=0.2, head_length=0.3, length_includes_head=True)

    for node, pos in node_positions.items():
        plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # Use marker='o' for circle markers
        plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        
    plt.axis('off')
    plt.gca().set_facecolor('w')
    plt.savefig('Plots/'+tree+'_'+layout+'_plot.png')
    plt.show()
    
    
#show_graph('Datasets/LesMiserables.dot','tree_layout1',(18,18))
#show_graph('Datasets/LesMiserables.dot','radial',tree = 'DFS')
#show_graph12("test_force.dot", layout="layered_ish", tree="BFS")

### Resolve Cycles

In [205]:
def child_nodes(edges,node):  #f_childs
    '''Returns the count of child nodes [0] and a list of them [1] for all nodes
    ''' 
    node_child_count = 0
    node_childs = []
    for edge in edges:
        if type(edges)==pydot.core.Edge:
            source = edge.get_source()
            destination = edge.get_destination()
        else:
            source = edge[0]
            destination = edge[1]
        if source == node:
            node_child_count += 1 
            node_childs.append(destination) 
    return node_child_count, node_childs


def parent_nodes(edges,node): #f_parents
    '''Returns the count of parent nodes [0] and a list of them [1] for all nodes
    '''
    node_parent_count = 0
    node_parents = []
    for edge in edges:
        if type(edges)==pydot.core.Edge:
            source = edge.get_source()
            destination = edge.get_destination()
        else:
            source = edge[0]
            destination = edge[1]
        if destination == node:
            node_parent_count += 1 
            node_parents.append(source)
    return node_parent_count, node_parents
#edges = [(0, 1), (1, 2), (2, 3), (3, 0), (0, 4), (1, 5), (2, 6), (3, 7), (4, 5), (5, 6), (6, 7), (7, 4), (4, 8), (5, 9)]
#print(list(get_data("test_force.dot")[1]))
#print(parent_nodes(get_data("test_force.dot")[1],'0')[1])
#parent_nodes(edges,node)


In [206]:
def sink_node(edges,node):
    #no outgoing edges
    return child_nodes(edges,node)[0] == 0

def source_node(edges,node):
    #no incoming edges
    return parent_nodes(edges,node)[0] == 0

In [207]:
def heuristic(file):
    '''returns a list with the edges that need to be reversed to eliminate cycles
    '''
    nodes, edges, node_list, edge_list, weights = get_data(file)[0], get_data(file)[1], get_data(file)[2], get_data(file)[3], get_data(file)[4]
    V = node_list
    e = edge_list
    A = set()
    invert = []
    #N->(v): childs
    #N<-(v): parents
    #while V != []:
    sink_nodes = [node for node in V if sink_node(e, node) == True] #no outgoing
    source_nodes = [node for node in V if source_node(e, node) == True] #no incoming
    isolated_nodes = [node for node in V if source_node(e, node) == True and sink_node(e, node) == True]
    
    while V != []:
        sink_nodes = [node for node in V if sink_node(e, node) == True]
        #print(sink_nodes)
        while sink_nodes != []:
            V.remove(sink_nodes[0])
            parents = parent_nodes(e,sink_nodes[0])[1]
            for parent in parents:
                edge = (parent,sink_nodes[0])
                A.add(edge)
                e.remove(edge)
            sink_nodes = [node for node in V if sink_node(e, node) == True]
        
        isolated_nodes = [node for node in V if source_node(e, node) == True and sink_node(e, node) == True]
        for node in isolated_nodes:
            V.remove(node)

        source_nodes = [node for node in V if source_node(e, node) == True] #no incoming
        while source_nodes != []:
            V.remove(source_nodes[0])
            childs = child_nodes(e,source_nodes[0])[1]
            for child in childs:
                edge = (source_nodes[0],child)
                A.add(edge)
                e.remove(edge)
                #edge = pydot.Edge( child,source_nodes[0])
                #e.remove(edge)
            source_nodes = [node for node in V if source_node(e, node) == True] #no incoming
        if V!=[]:
            maximal = max([(v,child_nodes(e,v)[0]-parent_nodes(e,v)[0]) for v in V],key=lambda x: x[1])
            #maximal = [(v,child_nodes(e,v)[0]-parent_nodes(e,v)[0]) for v in V]#,key=lambda x: x[1]
            #print(maximal)
            #break
            childs = child_nodes(e,maximal[0])[1]
            #print('node:',maximal[0],'maximal:',maximal[1])
            #print('childs', childs) 
            
            for child in childs:
                edge = (maximal[0],child)
                A.add(edge)
                e.remove(edge)
            parents = parent_nodes(e,maximal[0])[1]
            for parent in parents:
                edge = (parent,maximal[0])
                invert.append(edge)
                e.remove(edge)
            V.remove(maximal[0])
    return invert

def acyclic_graph(file,new_file):
    '''Checks whether the graph insidde file has cycles, and if so, it eliminates them and 
    creates a new file with the acyclic graph
    '''
    invert = heuristic(file)
    if invert !=[]:
        node_list, edge_list, weights = get_data(file)[2], get_data(file)[3], get_data(file)[4]
        for edge in invert:
            edge_list.remove(edge)
            edge_list.append((edge[1],edge[0]))
        #create_pydot_graph(nodes, edges, filename)
        return create_pydot_graph(node_list, edge_list, new_file,weights)
    return file
        
#print(heuristic('Datasets/smallnet.dot'))
acyclic_graph('heuristic.dot','heuristic_acyc.dot')
#show_graph12("test_force.dot", layout="layered_ish", tree="BFS")


### Layer Assignment

In [340]:
def get_edges(edges, node):
    edge_list = []
    for edge in edges:
        source = edge[0]
        dest = edge[1]
        if source == node:
            edge_list.append(dest)
    return edge_list

def get_layer_pos(matrix, element):
    '''Returns the layer and the position inside the layer of a certain element
    '''
    for layer, row in enumerate(matrix):
        for pos, value in enumerate(row):
            if value == element:
                return layer, pos 
    return None #if it doesnt find the element

def dummy(layers,edges):
    new_edges = edges
    layers_dict = {}
    for i,item in enumerate(layers):
        layers_dict[i] = item

    #adding the dummies
    for layer in layers_dict:
        for node in layers_dict[layer]:
            #d_edge_list = get_edges(edges, node)
            for dest in get_edges(edges, node):
                step = 1
                while dest not in layers_dict[layer + step] :
                    #dummy = str('l'+str(layer+step)+'_'+node+'_'+dest)
                    layers[layer+step].append(str('l'+str(layer+step)+'_'+node+'_'+dest))
                    step += 1

    dummies = [node for layer in layers for node in layer if node[0]=='l']
    max_layer = {dummy.split('_',1)[1]:get_layer_pos(layers,dummy.split('_')[2])[0] for dummy in dummies} #{'2_1': 3, '7_8': 3, '12_9': 2, '12_10': 2, '4_8': 3, '5_8': 3}
    min_layer = {dummy.split('_',1)[1]:get_layer_pos(layers,dummy.split('_')[1])[0] for dummy in dummies} #{'2_1': 0, '7_8': 0, '12_9': 0, '12_10': 0, '4_8': 1, '5_8': 1}
    #print(min_layer)

    #adding dummies' edges
    #print(dummies) #['l1_2_1', 'l1_7_8', 'l1_12_9', 'l1_12_10', 'l2_2_1', 'l2_7_8', 'l2_4_8', 'l2_5_8']
    for node in dummies:
        layer = int(node.split('_',1)[0][1:])
        edge_ish = node.split('_',1)[1]
        
        #source node -> first dummy
        if layer == int(1+min_layer[edge_ish]):# and layer != int(max_layer[edge_ish]-1):
            # print('MIN')
            # print('node:',node,'layer:',layer,'edge:',edge_ish,'min:',min_layer[edge_ish],'max:',max_layer[edge_ish])
            new_edges.append((node.split('_')[1],node)) #source, node: source ->node.split('_')[1]
        #last dummy -> dest
        if layer == int(max_layer[edge_ish]-1): #and layer != int(1+min_layer[edge_ish]):
            # print('MAX')
            # print('node:',node,'layer:',layer,'edge:',edge_ish,'min:',min_layer[edge_ish],'max:',max_layer[edge_ish])
            new_edges.append((node,node.split('_')[2])) #node, dest: dest ->node.split('_')[2]
        #dummy -> dummy
        if layer > int(1+min_layer[edge_ish]):#!= int(1+min_layer[edge_ish]) or layer != int(max_layer[edge_ish]-1):
            #print('NONE')
            #print('node:',node,'layer:',layer,'edge:',edge_ish,'min:',min_layer[edge_ish],'max:',max_layer[edge_ish])
            source = str('l'+str(int(layer-1))+'_'+edge_ish) 
            new_edges.append((source,node))
        
    remove_edges = []
    for edge in max_layer:
        new_edges.remove((edge.split('_')[0], edge.split('_')[1])) #original source,final dest
        remove_edges.append((edge.split('_')[0], edge.split('_')[1]))
             
    return layers,new_edges

def tomas_layer_assignment(file):
    '''Layer assignment with heigh Optimization
    '''
    nodes, edges, node_list, edge_list, weights = get_data(file)[0], get_data(file)[1], get_data(file)[2], get_data(file)[3], get_data(file)[4]
    layers = []
    S = [node for node in node_list if source_node(edge_list,node) == True]

    e = edge_list
    while S != []:
        layers.append(S)
        del_edges = []
        for node in S:
            new_edges = [edge for edge in e if edge[0]==node]
            for edge in new_edges:
                del_edges.append(edge)
        for edge in del_edges:
            e.remove(edge)
        node_list = [node for node in node_list if node not in S]
        S = [node for node in node_list if source_node(e,node) == True]

    final_layers = dummy(layers,get_data(file)[3])

    return final_layers[0],final_layers[1]

ans = tomas_layer_assignment('heuristic_acyc.dot')
#ans[1]

### Cross Minimization

In [338]:
def c_uv(u,v,edges,layer):
    '''u and v are nodes from layer i+1, and we want to see the number of crossing when we look at they're parents
    in layer i
    '''
    u_parents = [(parent,layer.index(parent)) for parent in layer if parent in parent_nodes(edges,u)[1]]
    v_parents = [(parent,layer.index(parent)) for parent in layer if parent in parent_nodes(edges,v)[1]]
    ans = 0
    for u_parent in u_parents:
        for v_parent in v_parents:
            if u_parent[0] != v_parent[0] and u_parent[1] > v_parent[1]:
                ans += 1
    return ans#u_parents, v_parents

nodes, edges, node_list, edge_list, weights = get_data('heuristic_acyc.dot')[0], get_data('heuristic_acyc.dot')[1], get_data('heuristic_acyc.dot')[2], get_data('heuristic_acyc.dot')[3], get_data('heuristic_acyc.dot')[4]
#print(edge_list)
layers = tomas_layer_assignment('heuristic_acyc.dot')[0]
edge_list = tomas_layer_assignment('heuristic_acyc.dot')[1]
#print(c_uv('5','l1_2_1',edge_list, layers[0]))

def cr(edges,layer1,layer2):
    '''Returns the total number of crossing between layer1 and layer2
    '''
    ans = 0
    for u in layer2:
        for v in layer2:
            if layer2.index(v)>layer2.index(u):
                ans+= c_uv(u,v,edges,layer1)
    return ans

#print(cr(edge_list,layers[2],layers[3]))


def bary(node,edges,layers,layer):
    ans = 0
    parents = parent_nodes(edges,node)[1]
    #print('node-',node, 'parents',parents)
    for parent in parents:
        #print(layer.index(parent))
        #print(parent)
        ans+= int(layers[layer-1].index(parent))
    den = 1
    if len(parents) != 0:
        den = len(parents)
    return ans/den

def it_cross_min(layers,edges,heur = 'Bary'):
    #1
    #random.shuffle(layers[0])
    print(layers[0])
    
    
    if heur == 'Bary':
        print('#######2######')
        for i in range(1,len(layers)-1):
            #change layer_i
            
            layer_i =layers[i]
            barycenters = sorted([(node,bary(node,edges,layers,i)) for node in layer_i], key=lambda x: x[1])
            #sorted_list = sorted(list_of_lists
            #layer_i_1 =layers[i+1]
            print(i,barycenters)
            layers[i] = [node[0] for node in barycenters]
            #crs_2.append(cr(edges,layer_i,layer_i_1))
        print('###############')
        #3
        print('layers',len(layers))
        print('#######3######')
        for i in range(1,len(layers)-1,-1):
            layer_i =layers[i]
            print(i)
            #change layer_i_1
            #layer_i_1 =layers[i+1]
            barycenters = sorted([(node,bary(node,edges,layers,i)) for node in layer_i], key=lambda x: x[1])
            print(i,barycenters)
            layers[i] = [node[0] for node in barycenters]
            #crs_2.append(cr(edges,layer_i,layer_i_1))


# print(layers)
# print(edge_list)
print(it_cross_min(layers,edge_list) )

        

['2', '7', '12']
#######2######
1 [('l1_2_1', 0.0), ('4', 1.0), ('5', 1.0), ('l1_7_8', 1.0), ('11', 2.0), ('l1_12_9', 2.0), ('l1_12_10', 2.0)]
2 [('l2_2_1', 0.0), ('l2_4_8', 1.0), ('3', 1.5), ('l2_5_8', 2.0), ('9', 3.0), ('l2_7_8', 3.0), ('6', 4.0), ('10', 4.0)]
###############
layers 4
#######3######
None


In [292]:
import random

# Original list
original_list = [1, 2, 3, 4, 5]

# Randomly reorder the list
random.shuffle(original_list)

# Print the randomly reordered list
print("Randomly reordered list:", original_list)


Randomly reordered list: [1, 2, 4, 5, 3]


In [210]:
def fruchterman_reingold(file, max_ites=1000, plot='No', c=2, dim =20, convergence_threshold=0.005): 
    nodes, edges, node_list, edge_list, weights = get_data(file)[0], get_data(file)[1], get_data(file)[2], get_data(file)[3], get_data(file)[4]
    node_positions = {node.get_name(): (random.uniform(0, 50), random.uniform(0, 50)) for node in nodes}

    l = c*math.sqrt(dim*dim/len(node_list)) 
    print(l)
    l_changes = []
    ite = 0
    while ite < max_ites:
        changes = []
        for u in nodes:
            force_x = 0
            force_y = 0
            pu = node_positions[u.get_name()]

            for v in nodes:
                if u.get_name() != v.get_name():
                    pv = node_positions[v.get_name()]
                    dist = math.sqrt((pu[0]-pv[0])**2 + (pu[1]-pv[1])**2)
                    if dist == 0:
                        dist = 1

                    # Spring
                    if (u.get_name(), v.get_name()) in edge_list or (v.get_name(), u.get_name()) in edge_list:
                        constant = dist**2/l
                        print(constant)
                        force_x += (pv[0]-pu[0])*constant/dist
                        force_y += (pv[1]-pu[1])*constant/dist
                    # Repulsion
                    
                    constant = l**2/dist
                    print(constant)
                    force_x += (pu[0]-pv[0])*constant/dist
                    force_y += (pu[1]-pv[1])*constant/dist
                changes.append(math.sqrt((force_x*0.01)**2 + (force_y*0.01)**2))
                new_x = pu[0] + force_x*0.01
                new_y = pu[1] + force_y*0.01
                node_positions[u.get_name()] = (new_x,new_y)

        average_movement = sum(changes) / len(changes)
        l_changes.append(average_movement)
        
        
        if average_movement < convergence_threshold:
            print("convergence signal!!!", ite) 
            break
        
        ite += 1

    if plot != 'No':
        plt.figure(figsize=(18, 18))         
        for edge in edges:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
            start_pos = node_positions[start_node]
            end_pos = node_positions[end_node]
            plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 

        for node, pos in node_positions.items():
            plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # vse marker='o' for circle markers
            plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        #plt.axis('off')
        plt.gca().set_facecolor('w')
        #plt.savefig('Plots/'+tree+'_'+layovt+'_plot.png')
        plt.show()


    return node_positions, l_changes


#l_f_v = fruchterman_reingold('Datasets/LesMiserables.dot', max_ites=1000, plot='Yes')

In [211]:
def working_spring_embedder(file, ite, plot='No', c_spring=2, c_rep=1, l=1):
    nodes, edges, node_list, edge_list, weights = get_data(file)[0], get_data(file)[1], get_data(file)[2], get_data(file)[3], get_data(file)[4]
    node_positions = {node.get_name(): (random.uniform(0, 50), random.uniform(0, 50)) for node in nodes} #Randomly initialize the positions of nodes.

    l_changes = []
    for _ in range(ite): #here maybe using convergence criteria instead of implementing  a certain number of interactions
        changes = []
        for u in nodes:
            force_x = 0
            force_y = 0
            pu = node_positions[u.get_name()]

            for v in nodes:
                if u.get_name() != v.get_name():
                    pv = node_positions[v.get_name()]
                    dist = math.sqrt((pu[0]-pv[0])**2 + (pu[1]-pv[1])**2)
                    if dist == 0:
                        dist = 1

                    # Spring
                    if (u.get_name(), v.get_name()) in edge_list or (v.get_name(), u.get_name()) in edge_list:
                        constant = c_spring*math.log(dist)/l
                        force_x += (pv[0]-pu[0])*constant/dist
                        force_y += (pv[1]-pu[1])*constant/dist
                    # Repulsion
                    else:
                        constant = c_rep/dist**2
                        force_x += (pu[0]-pv[0])*constant/dist
                        force_y += (pu[1]-pv[1])*constant/dist
            changes.append(math.sqrt((force_x*0.1)**2 + (force_y*0.1)**2))
            #new_x = pu[0] + force_x*0.1 #The learning rate controls the step size
            #new_y = pu[1] + force_y*0.1
            new_x = pu[0] + force_x*0.01 #The learning rate controls the step size
            new_y = pu[1] + force_y*0.01
            node_positions[u.get_name()] = (new_x,new_y)
        l_changes.append(sum(changes)/len(changes))
    if plot != 'No':
        plt.figure(figsize=(18, 18))         
        for edge in edges:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
            start_pos = node_positions[start_node]
            end_pos = node_positions[end_node]
            plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 

        for node, pos in node_positions.items():
            plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # vse marker='o' for circle markers
            plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        #plt.axis('off')
        plt.gca().set_facecolor('w')
        #plt.savefig('Plots/'+tree+'_'+layovt+'_plot.png')
        plt.show()

    return node_positions, l_changes

#l_f_v = working_spring_embedder('Datasets/LesMiserables.dot',ite = 1000,plot='Yes')#,c_spring=2, c_rep=1,l=1)

# Using Convergence Criteria?

In [212]:
def spring_embedder14(file, ite, plot='No', c_spring=2, c_rep=1, l=1):
    nodes, edges, node_list, edge_list, weights = get_data(file)[0], get_data(file)[1], get_data(file)[2], get_data(file)[3], get_data(file)[4]

    # Initialize random positions for nodes
    node_positions = {node.get_name(): (random.uniform(0, 50), random.uniform(0, 50)) for node in nodes}

    # Main loop for layout optimization
    l_changes = []
    for _ in range(ite):
        changes = []
        for u in nodes:
            force_x = 0
            force_y = 0
            pu = node_positions[u.get_name()]

            for v in nodes:
                if u.get_name() != v.get_name():
                    pv = node_positions[v.get_name()]
                    dist = math.sqrt((pu[0]-pv[0])**2 + (pu[1]-pv[1])**2)
                    if dist == 0:
                        dist = 1

                    # Spring force
                    if (u.get_name(), v.get_name()) in edge_list or (v.get_name(), u.get_name()) in edge_list:
                        constant = c_spring*math.log(dist/l)
                        force_x += (pv[0]-pu[0])*constant/dist
                        force_y += (pv[1]-pu[1])*constant/dist
                    # Repulsion force
                    else:
                        constant = c_rep/dist**2
                        force_x += (pu[0]-pv[0])*constant/dist
                        force_y += (pu[1]-pv[1])*constant/dist
            changes.append(math.sqrt((force_x*0.1)**2 + (force_y*0.1)**2))
            # Update node positions based on forces
            new_x = pu[0] + force_x*0.1
            new_y = pu[1] + force_y*0.1
            node_positions[u.get_name()] = (new_x,new_y)
        l_changes.append(max(changes))
    
    if plot != 'No':
        plt.figure(figsize=(18, 18))         
        for edge in edges:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
            start_pos = node_positions[start_node]
            end_pos = node_positions[end_node]
            plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 

        for node, pos in node_positions.items():
            plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # vse marker='o' for circle markers
            plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        #plt.axis('off')
        plt.gca().set_facecolor('w')
        #plt.sauefig('Plots/'+tree+'_'+layovt+'_plot.png')
        plt.show()

    return node_positions, l_changes

#l_f_v = spring_embedder14('Datasets/LesMiserables.dot',ite = 1000,plot='Yes')#,c_spring=2, c_rep=1,l=1)

In [213]:
def spring_embedder1(file,ite,plot = 'No',c_spring=2,c_rep=1,l=1):
    #l(e) --> ideal spring lenght for edge e
    #pv = (xv, yv) -->position of node v
    #||pu - pv|| -->euclidean distance between u and v
    #pupv --> unit vecor pointing from u to v
    #Values are appropiate for most graphs
    #c_srping = 2, l(e)=l= 1, c_rep = 1

    nodes,edges,node_list,edge_list,weights = get_data(file)[0],get_data(file)[1],get_data(file)[2],get_data(file)[3],get_data(file)[4]
    w=1
    initial_positions = {node.get_name():(int(random.uniform(0, 50)),int(random.uniform(0, 50))) for node in nodes}
    # forces = {}
    # for u in nodes:
    #     for v in nodes:
    l_f_v = []
    for i in range(ite):
        #print('o')
        for u in nodes:
            forces = []
            pu = initial_positions[u.get_name()]
            for v in nodes:
                if u.get_name() != v.get_name():
    
                    pv = initial_positions[v.get_name()]
                    dist = math.sqrt((pu[0]-pv[0])**2+(pu[1]-pv[1])**2)

                    if dist ==0:
                        dist = 1
                    pupv = ((pv[0]-pu[0])/dist,(pv[1]-pu[1])/dist)
                    #spring
                    if (u.get_name(),v.get_name()) in edge_list or (v.get_name(),u.get_name()) in edge_list:
                        constant = c_spring*math.log(dist)/l
                        force = (pupv[0]*constant*pupv[0], pupv[1]*constant*pupv[1])
                        forces.append(force)
                    #rep
                    else:
                        constant = c_rep/dist**2
                        force = (pupv[0]*constant*pupv[0], pupv[1]*constant*pupv[1])
                        forces.append(force)
            force_x = sum([force[0] for force in forces])
            force_y = sum([force[1] for force in forces])
            f_v = (force_x,force_y)
            l_f_v.append(math.sqrt(f_v[0]**2+f_v[1]**2))
            
            new_x1 = initial_positions[u.get_name()][0]+force_x*0.2
            new_y1 = initial_positions[u.get_name()][1]+force_y*0.2
            initial_positions[u.get_name()] = (new_x1,new_y1)

            # new_x2 = initial_positions[v.get_name()][0]-force_x*0.8
            # new_y2 = initial_positions[v.get_name()][1]-force_y*0.8
            # initial_positions[v.get_name()] = (new_x2,new_y2)
    #f_springs = [force for force]
    #return forces
    #return sum(forces)
    if plot != 'No':
        plt.figure(figsize=(18, 18))         
        for edge in edges:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
            start_pos = initial_positions[start_node]
            end_pos = initial_positions[end_node]
            plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 

        for node, pos in initial_positions.items():
            plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # Use marker='o' for circle markers
            plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        plt.axis('off')
        plt.gca().set_facecolor('w')
        #plt.savefig('Plots/'+tree+'_'+layout+'_plot.png')
        plt.show()

    return max(l_f_v)


#spring_embedder1('test_force.dot',1000,plot='t')

In [214]:
def spring_embedder(file, ite, epsilon, plot='No', c_spring=2, c_rep=1, l=1,delta=0.1):
    '''Implementation of the baasic spring_embedder
    file: .dot file with the data
    ite: number of iterations that we want to repeate the computation of new positions for each node
    epsilon: parameteres used to also tell when to stop, it corresponds to the maximum value of the forces
    c_spring: constant for computing the force between adjacent nodes
    c_rep: constant for computing the force between NON-adjacent nodes
    l(e): ideal spring lenght for edge e
    delta: step for updating the node position with delta*force
    '''
    nodes, edges, node_list, edge_list, weights = get_data(file)[0], get_data(file)[1], get_data(file)[2], get_data(file)[3], get_data(file)[4]   
    node_positions = {node.get_name(): (random.uniform(0, 50), random.uniform(0, 50)) for node in nodes}
    w=1
    l_f_v = {node.get_name():1 for node in nodes}
    i=0
    #max_changes = [50]
    while i<ite and max(l_f_v.values()) > epsilon:# and min(max_changes)>1:
        
        for v in nodes:
            changes = []
            force_x = 0
            force_y = 0
            pv = node_positions[v.get_name()]

            for u in nodes:
                if v.get_name() != u.get_name():
                    pu = node_positions[u.get_name()]
                    #dist from pu to pv is = to dist from pv to pu
                    dist = math.sqrt((pv[0]-pu[0])**2 + (pv[1]-pu[1])**2)
                    if dist == 0:
                        dist = 1

                    #spring
                    if (v.get_name(), u.get_name()) in edge_list or (u.get_name(), v.get_name()) in edge_list:
                        constant = c_spring*math.log(dist/l)
                        
                    #rep
                    else:
                        constant = c_rep/(dist**2)
                    
                    force_x += (pu[0]-pv[0])/dist*constant
                    force_y += (pu[1]-pv[1])/dist*constant
                    #changes.append(math.sqrt((force_x * delta)**2+(force_y * delta)**2))
            #l_f_v[v.get_name()] = math.sqrt(force_x**2+force_y**2)
            
            
            #change = math.sqrt((force_x * delta)**2+(force_y * delta)**2)
            #max_changes.append(sum(changes)/len(changes))

            v_new_x = pv[0] + force_x * delta
            v_new_y = pv[1] + force_y * delta
            node_positions[v.get_name()] = (v_new_x, v_new_y)
        
        i+=1

    if plot != 'No':
        plt.figure(figsize=(18, 18))         
        for edge in edges:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(str(edge.get_attributes())).rfind(':')
            if int(position) != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w/10
            start_pos = node_positions[start_node]
            end_pos = node_positions[end_node]
            plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-',linewidth=w,zorder = 1) 

        for node, pos in node_positions.items():
            plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder =2)  # vse marker='o' for circle markers
            plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        #plt.axis('off')
        plt.gca().set_facecolor('w')
        #plt.sauefig('Plots/'+tree+'_'+layovt+'_plot.png')
        plt.show()
    return node_positions#,l_f_v,i,max_changes
#l_f_v = spring_embedder('Datasets/LesMiserables.dot',ite = 1000,epsilon =0.01,plot='Yes')#,c_spring=2, c_rep=1,l=1)
#l_f_v = spring_embedder('test_force.dot',100,0.01,plot='N',c_spring=1.5, c_rep=1.5,l=1)
#l_f_v[1]

In [215]:
def spring_embedder12(file, ite, epsilon, plot='No', c_spring=2, c_rep=1, l=1, delta=0.4):
    nodes, edges, node_list, edge_list, weights = get_data(file)[0], get_data(file)[1], get_data(file)[2], get_data(file)[3], get_data(file)[4]   
    node_positions = {node.get_name(): (random.uniform(0, 50), random.uniform(0, 50)) for node in nodes}
    l_f_v = {node.get_name(): 1 for node in nodes}
    i = 0

    while i < ite and max(l_f_v.values()) > epsilon:
        l_f_v = {}
        for v in nodes:
            force_x = 0
            force_y = 0
            pv = node_positions[v.get_name()]
            l_f_v[v.get_name()] = 0

            for u in nodes:
                if v.get_name() != u.get_name():
                    pu = node_positions[u.get_name()]
                    dist = math.sqrt((pv[0] - pu[0]) ** 2 + (pv[1] - pu[1]) ** 2)
                    if dist == 0:
                        dist = 1

                    # Spring
                    if (v.get_name(), u.get_name()) in edge_list or (u.get_name(), v.get_name()) in edge_list:
                        constant = c_spring * math.log(dist / l)
                    # Repulsion
                    else:
                        constant = c_rep / (dist ** 2)

                    force_x += (pu[0] - pv[0]) * constant
                    force_y += (pu[1] - pv[1]) * constant

            # Normalize force vector
            force_magnitude = math.sqrt(force_x ** 2 + force_y ** 2)
            if force_magnitude != 0:
                force_x /= force_magnitude
                force_y /= force_magnitude

            l_f_v[v.get_name()] = force_magnitude

            v_new_x = pv[0] + force_x * delta
            v_new_y = pv[1] + force_y * delta
            node_positions[v.get_name()] = (v_new_x, v_new_y)
        
        i += 1

    if plot != 'No':
        plt.figure(figsize=(10, 10))
        for edge in edges:
            start_node = edge.get_source()
            end_node = edge.get_destination()
            position = str(edge.get_attributes()).rfind(':')
            if position != -1:
                w = int(str(edge.get_attributes())[position+3:-2])
                if w != 1:
                    w = w / 10
            start_pos = node_positions[start_node]
            end_pos = node_positions[end_node]
            plt.plot([start_pos[0], end_pos[0]], [start_pos[1], end_pos[1]], 'k-', linewidth=w, zorder=1)

        for node, pos in node_positions.items():
            plt.scatter(pos[0], pos[1], color='green', s=200, marker='o', zorder=2)
            plt.text(pos[0], pos[1], str(node), fontsize=10, ha='center', va='center')
        
        plt.gca().set_facecolor('w')
        plt.show()
    
    return node_positions

#spring_embedder12('Datasets/LesMiserables.dot',1000,0.01,plot='Yes')#,c_spring=2, c_rep=1,l=1)

In [216]:
#print(edges.get_values())
#for node in nodes:
 #   print("Node name:", node.get_name())
  #  print("Node label:", node.get_label())
   # print("Node attributes:", node.get_attributes())
    #print("Node shape:", node.get_shape())
    #print("Node style:", node.get_style())
    #print("Node position:", node.get_pos())
    
    
#for edge in edges:
 #   print("Source:", edge.get_source())
  #  print("Destination:", edge.get_destination())
    #print("Edge attributes:", edge.get_attributes())
    #position = str(str(edge.get_attributes())).rfind(':')
    #w = str(edge.get_attributes())[position+3:-2]
    #print(w)
    #print("Edge style:", edge.get_style())
    #print("Edge label:", edge.get_label())
    #print("Edge color:", edge.get_color())

# TRASH